In [1]:
#取得發文數大於5000的20個ID
import requests as req
from bs4 import BeautifulSoup
import time
import random
from random import choice
from random import sample

ID_href=[]
sID_href=[]
keep=True

#檢查此ID發文數是否大於5000
def getIDposts(IDhref):
    IDpage='https://www.pttweb.cc'+IDhref+'?t=article'
    source=req.get(IDpage).text
    root=BeautifulSoup(source,"lxml")
    postnum=root.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
    num=postnum[postnum.index("共")+1:len(postnum)-1]
    if(int(num)>5000):
        print(postnum)
    return int(num)

#尋找ID樣本
def choose_ID (currentpage_url):
    source=req.get(currentpage_url).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-box a')
    global ID_href
    global keep
    i=1
    for t in href:
        if(i%21==0):
            delay=[2,4,6]
            sec=choice(delay)
            print('等待'+str(sec)+'秒')
            time.sleep(sec)
        i=i+1
        if(getIDposts(t['href'])>5000):
            ID_href.append(t['href'])
        else:
            keep=False
            break

#下一頁
def nextpage_href(currentpage_url):
    source=req.get(currentpage_url).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-pagination a')
    #print(type(href))
    i=0
    for t in href:
        if i==10:
            target_url=t['href']
            break
        i+=1
    return target_url

#從取下的ID陣列中，隨機取20個ID
def getrandom_ID():
    global ID_href
    global sID_href
    sID_href=sample(ID_href,20)
    print(sID_href)  

#開跑
def run():
    page=1
    url='https://www.pttweb.cc/user?sort=article'
    while(keep):
        print('正在檢查第'+str(page)+'頁，URL為：'+url)
        choose_ID (url)
        url='https://www.pttweb.cc'+nextpage_href(url)
        page=page+1
        delay=[2,4,6]
        sec=choice(delay)
        print('等待'+str(sec)+'秒')
        time.sleep(sec)
    print('取得發文數大於5000的所有ID，現在隨機選出20個ID：')
    getrandom_ID()

run()

正在檢查第1頁，URL為：https://www.pttweb.cc/user?sort=article
作者 tjbulls 在 PTT 全部看板的發文, 共163147篇
作者 sp89005 在 PTT 全部看板的發文, 共66267篇
作者 childchen 在 PTT 全部看板的發文, 共47918篇
作者 zkow 在 PTT 全部看板的發文, 共46212篇


KeyboardInterrupt: 

In [108]:
#取得隨機選取的20個ID裡，留言數大於30的文章放進相應年份的陣列，最後再隨機取出每年20個文章的URL
import requests as req
from bs4 import BeautifulSoup
import time
import random
from random import choice
from random import sample
from collections import defaultdict
from fake_useragent import UserAgent
import re
#'/user/paraguay', '/user/hcc19880512'(都是灌水文), '/user/majaja5566', '/user/blueherobest', '/user/kitune', 
#'/user/cloud654', '/user/qscxz', '/user/moshenisshit', '/user/htx9', '/user/aisingioro', '/user/ayanami5566', 
#'/user/wl760713', '/user/ilovemumi', '/user/buslover', 
#'/user/geordie', '/user/an94mod0', '/user/better83214', '/user/mono5566', '/user/naruto', '/user/fantiida'
ID_href=['/user/j00788']

URL2022=[]
URL2021=[]
URL2020=[]
URL2019=[]
URL2018=[]
URL2017=[]
URL2016=[]
URL2015=[]
URL2014=[]
URL2013=[]
URL2012=[]
URL2011=[]
page=1

default_options = defaultdict(lambda: 4, {'2011': URL2011, '2012': URL2012, '2013': URL2013, '2014': URL2014,
                                          '2015': URL2015, '2016': URL2016, '2017': URL2017, '2018': URL2018,
                                          '2019': URL2019, '2020': URL2020, '2021': URL2021, '2022': URL2022})

#下一頁
def nextpage_href(currentpage_url):
    source=req.get(currentpage_url).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-pagination a')
    #print(type(href))
    i=0
    for t in href:
        if i==10:
            target_url=t['href']
            break
        i+=1
    return target_url

#取得該ID發文留言大於30的文章
def getposts(currentpage_url):
    user_agent = UserAgent()
    source=req.get(currentpage_url, headers={ 'user-agent': user_agent.random }).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-left a')
    posttimes=root.find_all("span", {"class":"thread-posttime"})
    if(len(posttimes)==0):
        print('『沒正確爬到資料，再爬一次！』')
        getposts(currentpage_url)
        return
    comments=root.find_all("span", {"class":{"thread-popularity px-2"}})
    postdate=[]
    posturl=[]
    p=0
    for t in href:
        a=t['href']
        a=re.sub('[^/]','',a)        
        if(len(a)>2):    #判斷href中含有的/符號是否大於2，若大於2，及為目標href
            posturl.append(t['href'])
        p=p+1
    for i in range(0,len(posttimes),2):
        postdate.append(posttimes[i].text)
    p=0
    for comment in comments:
        a=comment.text
        a=a[0:a.index("留")-1]
        year=postdate[p]
        year=year[0:4]
        if(int(year)<2011):
            print('https://www.pttweb.cc'+posturl[p])
            print('『此發文久於指定年份，停止爬蟲此ID的作業！』')
            return False
        if(int(a)>30):
            print('https://www.pttweb.cc'+posturl[p])
            default_options[year].append('https://www.pttweb.cc'+posturl[p])
        p=p+1

#開跑
def run(): 
    global ID_href
    global page
    for i in range(len(ID_href)):
        ID=ID_href[i]
        ID=ID[6:len(ID)]
        print('現在正在爬取ID：'+ID)
        user_agent = UserAgent()
        url='https://www.pttweb.cc'+ID_href[i]+'?t=article'
        source=req.get(url, headers={ 'user-agent': user_agent.random }).text
        root=BeautifulSoup(source,"lxml")
        postnum=root.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
        postnum=postnum[postnum.index("共")+1:len(postnum)-1]
        page=int(postnum)/10
        page=int(page)
        for i in range(page+1):
            print('現在檢查到第'+str((i+1))+'頁，URL為：'+url)
            if(getposts(url)==False):
                break;
            url='https://www.pttweb.cc'+nextpage_href(url)
            if((i+1)%10 == 0):
                delay=[3,6,9]
                sec=choice(delay)
                print('等待'+str(sec)+'秒')
                time.sleep(sec)
        page=0
        with open('test.txt','a') as f:
            f.write('ID：'+ID+'做完後的全部文章連結')
            f.write('\n')
            f.write('2021的文章：')
            f.write('[')
            for i in range(len(URL2021)):
                f.write('\n')
                f.write("'"+URL2021[i]+"',")
            f.write(']\n')
            f.write('2020的文章：')
            f.write('[')
            for i in range(len(URL2020)):
                f.write('\n')
                f.write("'"+URL2020[i]+"',")
            f.write(']\n')
            f.write('2019的文章：')
            f.write('[')
            for i in range(len(URL2019)):
                f.write('\n')
                f.write("'"+URL2019[i]+"',")
            f.write(']\n')
            f.write('2018的文章：')
            f.write('[')
            for i in range(len(URL2018)):
                f.write('\n')
                f.write("'"+URL2018[i]+"',")
            f.write(']\n')
            f.write('2017的文章：')
            f.write('[')
            for i in range(len(URL2017)):
                f.write('\n')
                f.write("'"+URL2017[i]+"',")
            f.write(']\n')
            f.write('2016的文章：')
            f.write('[')
            for i in range(len(URL2016)):
                f.write('\n')
                f.write("'"+URL2016[i]+"',")
            f.write(']\n')
            f.write('2015的文章：')
            f.write('[')
            for i in range(len(URL2015)):
                f.write('\n')
                f.write("'"+URL2015[i]+"',")
            f.write(']\n')
            f.write('2014的文章：')
            f.write('[')
            for i in range(len(URL2014)):
                f.write('\n')
                f.write("'"+URL2014[i]+"',")
            f.write(']\n')
            f.write('2013的文章：')
            f.write('[')
            for i in range(len(URL2013)):
                f.write('\n')
                f.write("'"+URL2013[i]+"',")
            f.write(']\n')
            f.write('2012的文章：')
            f.write('[')
            for i in range(len(URL2012)):
                f.write('\n')
                f.write("'"+URL2012[i]+"',")
            f.write(']\n')
            f.write('2011的文章：')
            f.write('[')
            for i in range(len(URL2011)):
                f.write('\n')
                f.write("'"+URL2011[i]+"',")
            f.write(']\n')
            f.write("-----------『測試結束！』-----------")
            f.write('\n')
        print('ID：'+ID+'發文中留言數大於30的文章已經爬蟲檢查完畢')
        print('目前陣列中有多少post：')
        print('2021年有：'+str(len(URL2021))+'篇')
        print('2020年有：'+str(len(URL2020))+'篇')
        print('2019年有：'+str(len(URL2019))+'篇')
        print('2018年有：'+str(len(URL2018))+'篇')
        print('2017年有：'+str(len(URL2017))+'篇')
        print('2016年有：'+str(len(URL2016))+'篇')
        print('2015年有：'+str(len(URL2015))+'篇')
        print('2014年有：'+str(len(URL2014))+'篇')
        print('2013年有：'+str(len(URL2013))+'篇')
        print('2012年有：'+str(len(URL2012))+'篇')
        print('2011年有：'+str(len(URL2011))+'篇')

run()

現在正在爬取ID：j00788
現在檢查到第1頁，URL為：https://www.pttweb.cc/user/j00788?t=article
現在檢查到第2頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=1
現在檢查到第3頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=2
https://www.pttweb.cc/bbs/Comic/M.1506314366.A.AB6
https://www.pttweb.cc/bbs/KoreaStar/M.1499341895.A.93C
現在檢查到第4頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=3
https://www.pttweb.cc/bbs/SS501/M.1486813292.A.C61
現在檢查到第5頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=4
https://www.pttweb.cc/bbs/SS501/M.1459614238.A.ECF
現在檢查到第6頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=5
https://www.pttweb.cc/bbs/KoreaDrama/M.1458575924.A.F1C
https://www.pttweb.cc/bbs/SS501/M.1456546789.A.236
https://www.pttweb.cc/bbs/SS501/M.1455767700.A.19D
https://www.pttweb.cc/bbs/book/M.1440905524.A.0B0
現在檢查到第7頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=6
https://www.pttweb.cc/bbs/KoreanPop/M.1427954482.A.F79
現在檢查到第8頁，URL為：https://www.pttweb.cc/user/j00788?t=article&page=7


In [85]:
import requests as req
from bs4 import BeautifulSoup
import concurrent.futures
from fake_useragent import UserAgent

def nextpage_href(currentpage_url):
    user_agent = UserAgent()
    source=req.get(currentpage_url, headers={ 'user-agent': user_agent.random }).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-pagination a')
    #print(type(href))
    i=0
    for t in href:
        if i==10:
            target_href=t['href']
            break
        i+=1
    return target_href

def getposts(currentpage_url ,currentpage_page):
    user_agent = UserAgent()
    source=req.get(currentpage_url, headers={ 'user-agent': user_agent.random }).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-left a')
    posttimes=root.find_all("span", {"class":"thread-posttime"})
    if(len(posttimes)==0):
        print('『沒正確爬到資料，再爬一次！』')
        getposts(currentpage_url)
        return
    comments=root.find_all("span", {"class":{"thread-popularity px-2"}})
    postdate=[]
    posturl=[]
    p=0
    for t in href:
        a=t['href']
        a=re.sub('[^/]','',a)        
        if(len(a)>2):    #判斷href中含有的/符號是否大於2，若大於2，及為目標href
            posturl.append(t['href'])
        p=p+1
    for i in range(0,len(posttimes),2):
        postdate.append(posttimes[i].text)
    p=0
    print('現在檢查到第'+str(currentpage_page)+'頁，URL為：'+currentpage_url)
    for comment in comments:
        a=comment.text
        a=a[0:a.index("留")-1]
        year=postdate[p]
        year=year[0:4]
        if(int(year)<2011):
            print('https://www.pttweb.cc'+posturl[p])
            print('『此發文久於指定年份，停止爬蟲此ID的作業！』')
            return False
        if(int(a)>30):
            print('https://www.pttweb.cc'+posturl[p])
            default_options[year].append('https://www.pttweb.cc'+posturl[p])
        p=p+1



url='https://www.pttweb.cc/user/wl760713?t=article'
urls=[]
pages=[]
page = 1

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for i in range(100):
        for i in range(10):
            urls.append(url)
            pages.append(page)
            url='https://www.pttweb.cc'+nextpage_href(url)
            page=page+1
        executor.map(getposts, urls, pages)
        urls.clear()
        pages.clear()


現在檢查到第2頁，URL為：https://www.pttweb.cc/user/wl760713?t=article&page=1
https://www.pttweb.cc/bbs/C_Chat/M.1641043265.A.905
https://www.pttweb.cc/bbs/C_Chat/M.1641006375.A.A8C
https://www.pttweb.cc/bbs/C_Chat/M.1641001479.A.05F
現在檢查到第9頁，URL為：https://www.pttweb.cc/user/wl760713?t=article&page=8
https://www.pttweb.cc/bbs/C_Chat/M.1639843044.A.E81
https://www.pttweb.cc/bbs/C_Chat/M.1639823053.A.12C
https://www.pttweb.cc/bbs/C_Chat/M.1639819509.A.FF8
https://www.pttweb.cc/bbs/C_Chat/M.1639812310.A.0B3
https://www.pttweb.cc/bbs/C_Chat/M.1639797908.A.8E3
https://www.pttweb.cc/bbs/PuzzleDragon/M.1639797273.A.058
現在檢查到第7頁，URL為：https://www.pttweb.cc/user/wl760713?t=article&page=6
https://www.pttweb.cc/bbs/C_Chat/M.1640073302.A.5CC
https://www.pttweb.cc/bbs/C_Chat/M.1640005966.A.E4B
https://www.pttweb.cc/bbs/C_Chat/M.1640004285.A.04F
https://www.pttweb.cc/bbs/C_Chat/M.1640003376.A.3D7
https://www.pttweb.cc/bbs/C_Chat/M.1639998086.A.875
https://www.pttweb.cc/bbs/PuzzleDragon/M.1639990275.A.F27
現在檢查到第3

KeyboardInterrupt: 

In [124]:
import requests as req
from bs4 import BeautifulSoup
import time
import random
from random import choice
from random import sample
from collections import defaultdict
from fake_useragent import UserAgent
import re

URL2021=[]
URL2020=[]
URL2019=[]
URL2018=[]
URL2017=[]
URL2016=[]
URL2015=[]
URL2014=[]
URL2013=[]
URL2012=[]
URL2011=[]
default_options = defaultdict(lambda: 4, {'2011': URL2011, '2012': URL2012, '2013': URL2013, '2014': URL2014,
                                          '2015': URL2015, '2016': URL2016, '2017': URL2017, '2018': URL2018,
                                          '2019': URL2019, '2020': URL2020, '2021': URL2021})


year = "2021"

def loaddata(path):
    with open(path) as f:
        for line in f.readlines():
            string = line.replace('\n','')
            try:
                string = re.sub("'",'',string)
                string = re.sub(',','',string)
                string = string.replace('[','').replace(']','')
                if('https' not in string):
                    year = string[0:4]
                else:
                    default_options[year].append(string)
            except:
                print('false')
                pass
            continue

loaddata('totaldata.txt')

print('2021年有：'+str(len(URL2021))+'篇')
print('2020年有：'+str(len(URL2020))+'篇')
print('2019年有：'+str(len(URL2019))+'篇')
print('2018年有：'+str(len(URL2018))+'篇')
print('2017年有：'+str(len(URL2017))+'篇')
print('2016年有：'+str(len(URL2016))+'篇')
print('2015年有：'+str(len(URL2015))+'篇')
print('2014年有：'+str(len(URL2014))+'篇')
print('2013年有：'+str(len(URL2013))+'篇')
print('2012年有：'+str(len(URL2012))+'篇')
print('2011年有：'+str(len(URL2011))+'篇')

size = 100
sURL2021 = sample(list(URL2021),size)
sURL2020 = sample(list(URL2020),size)
sURL2019 = sample(list(URL2019),size)
sURL2018 = sample(list(URL2018),size)
sURL2017 = sample(list(URL2017),size)
sURL2016 = sample(list(URL2016),size)
sURL2015 = sample(list(URL2015),size)
sURL2014 = sample(list(URL2014),size)
sURL2013 = sample(list(URL2013),size)
sURL2012 = sample(list(URL2012),size)

with open('fianlresult(1000).txt','a') as f:
    f.write('2021的文章：\n')
    f.write('[')
    for i in range(len(sURL2021)):
        f.write("'"+sURL2021[i]+"'")
        if(i<len(sURL2021)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2020的文章：\n')
    f.write('[')
    for i in range(len(sURL2020)):
        f.write("'"+sURL2020[i]+"'")
        if(i<len(sURL2020)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2019的文章：\n')
    f.write('[')
    for i in range(len(sURL2019)):
        f.write("'"+sURL2019[i]+"'")
        if(i<len(sURL2020)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2018的文章：\n')
    f.write('[')
    for i in range(len(sURL2018)):
        f.write("'"+sURL2018[i]+"'")
        if(i<len(sURL2020)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2017的文章：\n')
    f.write('[')
    for i in range(len(sURL2017)):
        f.write("'"+sURL2017[i]+"'")
        if(i<len(sURL2017)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2016的文章：\n')
    f.write('[')
    for i in range(len(sURL2016)):
        f.write("'"+sURL2016[i]+"'")
        if(i<len(sURL2016)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2015的文章：\n')
    f.write('[')
    for i in range(len(sURL2015)):
        f.write("'"+sURL2015[i]+"'")
        if(i<len(sURL2015)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2014的文章：\n')
    f.write('[')
    for i in range(len(sURL2014)):
        f.write("'"+sURL2014[i]+"'")
        if(i<len(sURL2014)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2013的文章：\n')
    f.write('[')
    for i in range(len(sURL2013)):
        f.write("'"+sURL2013[i]+"'")
        if(i<len(sURL2013)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2012的文章：\n')
    f.write('[')
    for i in range(len(sURL2012)):
        f.write("'"+sURL2012[i]+"'")
        if(i<len(sURL2012)-1):
            f.write(',\n')
    f.write(']\n')
    f.write("-----------『測試結束！』-----------")

2021年有：8832篇
2020年有：9424篇
2019年有：9388篇
2018年有：8765篇
2017年有：7728篇
2016年有：4952篇
2015年有：1462篇
2014年有：806篇
2013年有：912篇
2012年有：181篇
2011年有：45篇


In [115]:
import requests as req
from bs4 import BeautifulSoup
import time
import random
from random import choice
from random import sample
from collections import defaultdict
from fake_useragent import UserAgent
import re

for i in range(i,10)

URL2021=[]
URL2020=[]
URL2019=[]
URL2018=[]
URL2017=[]
URL2016=[]
URL2015=[]
URL2014=[]
URL2013=[]
URL2012=[]
URL2011=[]
default_options = defaultdict(lambda: 4, {'2011': URL2011, '2012': URL2012, '2013': URL2013, '2014': URL2014,
                                          '2015': URL2015, '2016': URL2016, '2017': URL2017, '2018': URL2018,
                                          '2019': URL2019, '2020': URL2020, '2021': URL2021})


year = "2021"

def loaddata(path):
    with open(path) as f:
        for line in f.readlines():
            string = line.replace('\n','')
            try:
                string = re.sub("'",'',string)
                string = re.sub(',','',string)
                string = string.replace('[','').replace(']','')
                if('https' not in string):
                    year = string[0:4]
                else:
                    default_options[year].append(string)
            except:
                print('false')
                pass
            continue

loaddata('totaldata.txt')

print('2021年有：'+str(len(URL2021))+'篇')
print('2020年有：'+str(len(URL2020))+'篇')
print('2019年有：'+str(len(URL2019))+'篇')
print('2018年有：'+str(len(URL2018))+'篇')
print('2017年有：'+str(len(URL2017))+'篇')
print('2016年有：'+str(len(URL2016))+'篇')
print('2015年有：'+str(len(URL2015))+'篇')
print('2014年有：'+str(len(URL2014))+'篇')
print('2013年有：'+str(len(URL2013))+'篇')
print('2012年有：'+str(len(URL2012))+'篇')
print('2011年有：'+str(len(URL2011))+'篇')
sURL2021 = sample(list(URL2021),20)
sURL2020 = sample(list(URL2020),20)
sURL2019 = sample(list(URL2019),20)
sURL2018 = sample(list(URL2018),20)
sURL2017 = sample(list(URL2017),20)
sURL2016 = sample(list(URL2016),20)
sURL2015 = sample(list(URL2015),20)
sURL2014 = sample(list(URL2014),20)
sURL2013 = sample(list(URL2013),20)
sURL2012 = sample(list(URL2012),20)
sURL2011 = sample(list(URL2011),20)

with open('fianlresult.txt','a') as f:
    f.write('2021的文章：\n')
    f.write('[')
    for i in range(len(sURL2021)):
        f.write("'"+sURL2021[i]+"'")
        if(i<len(sURL2021)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2020的文章：\n')
    f.write('[')
    for i in range(len(sURL2020)):
        f.write("'"+sURL2020[i]+"'")
        if(i<len(sURL2020)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2019的文章：\n')
    f.write('[')
    for i in range(len(sURL2019)):
        f.write("'"+sURL2019[i]+"'")
        if(i<len(sURL2020)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2018的文章：\n')
    f.write('[')
    for i in range(len(sURL2018)):
        f.write("'"+sURL2018[i]+"'")
        if(i<len(sURL2020)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2017的文章：\n')
    f.write('[')
    for i in range(len(sURL2017)):
        f.write("'"+sURL2017[i]+"'")
        if(i<len(sURL2017)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2016的文章：\n')
    f.write('[')
    for i in range(len(sURL2016)):
        f.write("'"+sURL2016[i]+"'")
        if(i<len(sURL2016)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2015的文章：\n')
    f.write('[')
    for i in range(len(sURL2015)):
        f.write("'"+sURL2015[i]+"'")
        if(i<len(sURL2015)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2014的文章：\n')
    f.write('[')
    for i in range(len(sURL2014)):
        f.write("'"+sURL2014[i]+"'")
        if(i<len(sURL2014)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2013的文章：\n')
    f.write('[')
    for i in range(len(sURL2013)):
        f.write("'"+sURL2013[i]+"'")
        if(i<len(sURL2013)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2012的文章：\n')
    f.write('[')
    for i in range(len(sURL2012)):
        f.write("'"+sURL2012[i]+"'")
        if(i<len(sURL2012)-1):
            f.write(',\n')
    f.write(']\n')
    f.write('2011的文章：\n')
    f.write('[')
    for i in range(len(sURL2011)):
        f.write("'"+sURL2011[i]+"'")
        if(i<len(sURL2011)-1):
            f.write(',\n')
    f.write(']\n')
    f.write("-----------『測試結束！』-----------")

IndentationError: expected an indented block (<ipython-input-115-4182f82252aa>, line 14)

In [ ]:
with open('totaldata.txt','a') as f:
    f.write('2021的文章：\n')
    for i in range(len(URL2021)):
        f.write(URL2021[i]+'\n')
    f.write('2020的文章：\n')
    for i in range(len(URL2020)):
        f.write(URL2020[i]+'\n')
    f.write('2019的文章：\n')
    for i in range(len(URL2019)):
        f.write(URL2019[i]+'\n')
    f.write('2018的文章：\n')
    for i in range(len(URL2018)):
        f.write(URL2018[i]+'\n')
    f.write('2017的文章：\n')
    for i in range(len(URL2017)):
        f.write(URL2017[i]+'\n')
    f.write('2016的文章：\n')
    for i in range(len(URL2016)):
        f.write(URL2016[i]+'\n')
    f.write('2015的文章：\n')
    for i in range(len(URL2015)):
        f.write(URL2015[i]+'\n')
    f.write('2014的文章：\n')
    for i in range(len(URL2014)):
        f.write(URL2014[i]+'\n')
    f.write('2013的文章：\n')
    for i in range(len(URL2013)):
        f.write(URL2013[i]+'\n')
    f.write('2012的文章：\n')
    for i in range(len(URL2012)):
        f.write(URL2012[i]+'\n')
    f.write('2011的文章：\n')
    for i in range(len(URL2011)):
        f.write(URL2011[i]+'\n')
    f.write("-----------『測試結束！』-----------")

In [5]:
import requests as req
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
 
 
user_agent = UserAgent()
source = req.get(url='https://www.pttweb.cc/user/ivorysoap?t=article', headers={ 'user-agent': user_agent.random }).text
root=BeautifulSoup(source,"lxml")
href=root.select('div.e7-left a')
p=0
for t in href:
        if(p%2==0):
            print(t['href'])
        p=p+1

/bbs/Gossiping/M.1639446103.A.F86
/bbs/Gossiping/M.1639359023.A.927
/bbs/Gossiping/M.1639358015.A.0B3
/bbs/Gossiping/M.1639355519.A.64A
/bbs/Gossiping/M.1639354784.A.698
/bbs/Gossiping/M.1639354179.A.C23
/bbs/Gossiping/M.1639292623.A.36C
/bbs/Gossiping/M.1639291641.A.56B
/bbs/Gossiping/M.1639283300.A.2DF
/bbs/Gossiping/M.1639281547.A.E9D


In [11]:
from collections import defaultdict
from random import sample

URL2021=[]
URL2020=[]
URL2019=[]
URL2018=[]
URL2017=[]
URL2016=[]
URL2015=[]
URL2014=[]
URL2013=[]
URL2012=[]
URL2011=[]
keep=True
page=1

default_options = defaultdict(lambda: 4, {'2011': URL2011, '2012': URL2012, '2013': URL2013, '2014': URL2014,
                                          '2015': URL2015, '2016': URL2016, '2017': URL2017, '2018': URL2018,
                                          '2019': URL2019, '2020': URL2020, '2021': URL2021})
the_thing = default_options['2011']
default_options['2011'].append('start')
print(the_thing)
for i in range(11):
        year=2010+i
        a=default_options[str(year)]
        print(a)

def getrandom_post(): 
    for i in range(11):
        year=2010+i
        a=default_options[str(year)]
        b=sample(a,20)
        print(b) 
        
#getrandom_post()

['start']
4
['start']
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [35]:
#尋找下一頁的href的function
import requests as req
from bs4 import BeautifulSoup

url='https://www.pttweb.cc/user/cavendishjr?t=article&k1=1625711742000000000&k2=56603581'

def nextpage_href(currentpage_url):
    source=req.get(currentpage_url).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-pagination a')
    #print(type(href))
    i=0
    for t in href:
        if i==10:
            target_url=t['href']
            break
        i+=1
    return target_url
nextpage_href(url)

'/user/cavendishjr?t=article&k1=1625648279000000000&k2=56597066'

In [76]:
#尋找下一頁的href的function
import requests as req
from bs4 import BeautifulSoup


url='https://www.pttweb.cc/user/cavendishjr?t=article&k1=1625711742000000000&k2=56603581'

def nextpage_href(currentpage_url):
    source=req.get(currentpage_url).text
    root=BeautifulSoup(source,"lxml")
    href=root.select('div.e7-pagination a')
    #print(type(href))
    i=0
    for t in href:
        if i==10:
            target_url=t['href']
            break
        i+=1
    return target_url

nextpage_href(url)

'/user/cavendishjr?t=article&k1=1625648279000000000&k2=56597066'

In [59]:
#尋找這個頁面url所有文章的發文時間的function
import requests as req
from bs4 import BeautifulSoup





url='https://www.pttweb.cc/user/cavendishjr?t=article&k1=1625648279000000000&k2=56597066'

def post_time(currentpage_url):
    source=req.get(url).text
    root=BeautifulSoup(source,"lxml")
    posttime=root.select('span.thread-posttime')
    i=0
    for p in posttime:
        if(i%2 == 0):
            print(p.text)
        i = i + 1

post_time(url)

2021/07/07 16:57
2021/07/07 16:26
2021/07/07 15:05
2021/07/07 14:50
2021/07/07 14:28
2021/07/07 12:24
2021/07/07 12:07
2021/07/07 11:26
2021/07/06 20:08
2021/07/06 19:25


In [10]:
import requests as req
from bs4 import BeautifulSoup
import datetime,random

def randomtimes (start, end, n ) :
 frmt="%Y/%m/%d %H:%M"
 stime=datetime.datetime.strptime(start,frmt)
 etime=datetime.datetime.strptime(end, frmt)
 time_datetime=[random.random() * (etime - stime) + stime for _ in range(n)]
 time_str=[t.strftime(frmt) for t in time_datetime]
 return time_str

url='https://www.pttweb.cc/user/cavendishjr?t=article'
source=req.get(url).text
root=BeautifulSoup(source,"lxml")
posttimes=root.find_all("span",{"class":"thread-posttime"})
for i in range(0,20,2):
    print(posttimes[i].text)

a=randomtimes ('2021/10/05 16:27', '2021/10/5 16:27' , 1)
print(a[0])
for i in range(0,20,2):
    if(posttimes[i].text==a[0]):
        print(posttimes[i].text==a[0])
        break

2021/11/19 08:25
2021/11/19 08:21
2021/11/18 23:37
2021/11/18 23:32
2021/11/18 20:56
2021/11/18 20:39
2021/11/18 18:50
2021/11/18 18:40
2021/11/18 18:08
2021/11/18 12:14
2021/10/05 16:27


In [180]:
import requests as req
from bs4 import BeautifulSoup

url='https://www.pttweb.cc/user/cavendishjr?t=article&page=99'
source=req.get(url).text
root=BeautifulSoup(source,"lxml")
url=[]
for a in root.find_all("a", class_="router-link-active"):
    print(a['href'])
    url.append(a['href'])
print(url[16])

/user
/user/cavendishjr
/user/cavendishjr?t=article
/user/cavendishjr?t=message
/user/cavendishjr?t=nickname
/user/cavendishjr?t=article
/user/cavendishjr?t=article
/user/cavendishjr?t=article&page=98
/user/cavendishjr?t=article&page=95
/user/cavendishjr?t=article&page=96
/user/cavendishjr?t=article&page=97
/user/cavendishjr?t=article&page=98
/user/cavendishjr?t=article&page=-1
/user/cavendishjr?t=article
/user/cavendishjr?t=article&page=98
/user/cavendishjr?t=article&page=95
/user/cavendishjr?t=article&page=96
/user/cavendishjr?t=article&page=97
/user/cavendishjr?t=article&page=98
/user/cavendishjr?t=article&page=-1
/user/cavendishjr?t=article&page=96


In [157]:
import requests as req
from bs4 import BeautifulSoup

url='https://www.pttweb.cc/user/cavendishjr?t=article&k1=1625387184000000000&k2=56568070'
source=req.get(url).text
root=BeautifulSoup(source,"lxml")
url=[]
for a in root.find_all("a",  class_="router-link-active"):
    print(a['href'])
    url.append(a['href'])
print(url[11])

/user
/user/cavendishjr
/user/cavendishjr?t=article
/user/cavendishjr?t=message
/user/cavendishjr?t=nickname
/user/cavendishjr?t=article
/user/cavendishjr?t=article
/user/cavendishjr?t=article&k1=1625464131000000000&k2=56576671
/user/cavendishjr?t=article&k1=1625649573000000000&k2=56597214
/user/cavendishjr?t=article&k1=1625570736000000000&k2=56588899
/user/cavendishjr?t=article&k1=1625542155000000000&k2=56584847
/user/cavendishjr?t=article&k1=1625464131000000000&k2=56576671
/user/cavendishjr?t=article&k1=1625294939000000000&k2=56557370
/user/cavendishjr?t=article&k1=1625213576000000000&k2=56548305
/user/cavendishjr?t=article&k1=1625197758000000000&k2=56545665
/user/cavendishjr?t=article&k1=1625143735000000000&k2=56540578
/user/cavendishjr?t=article&k1=1625294939000000000&k2=56557370
/user/cavendishjr?t=article
/user/cavendishjr?t=article&k1=1625464131000000000&k2=56576671
/user/cavendishjr?t=article&k1=1625649573000000000&k2=56597214
/user/cavendishjr?t=article&k1=1625570736000000000&

In [178]:
import requests as req
from bs4 import BeautifulSoup

url='https://www.pttweb.cc/user/cavendishjr?t=article&k1=1625387184000000000&k2=56568070'
source=req.get(url).text
root=BeautifulSoup(source,"lxml")
href=root.select('div.e7-pagination a')
print(type(href))
for t in href:
    print(t['href'])

<class 'bs4.element.ResultSet'>
/user/cavendishjr?t=article
/user/cavendishjr?t=article&k1=1625464131000000000&k2=56576671
/user/cavendishjr?t=article&k1=1625649573000000000&k2=56597214
/user/cavendishjr?t=article&k1=1625570736000000000&k2=56588899
/user/cavendishjr?t=article&k1=1625542155000000000&k2=56584847
/user/cavendishjr?t=article&k1=1625464131000000000&k2=56576671
/user/cavendishjr?t=article&k1=1625294939000000000&k2=56557370
/user/cavendishjr?t=article&k1=1625213576000000000&k2=56548305
/user/cavendishjr?t=article&k1=1625197758000000000&k2=56545665
/user/cavendishjr?t=article&k1=1625143735000000000&k2=56540578
/user/cavendishjr?t=article&k1=1625294939000000000&k2=56557370
/user/cavendishjr?t=article&page=-1
/user/cavendishjr?t=article
/user/cavendishjr?t=article&k1=1625464131000000000&k2=56576671
/user/cavendishjr?t=article&k1=1625649573000000000&k2=56597214
/user/cavendishjr?t=article&k1=1625570736000000000&k2=56588899
/user/cavendishjr?t=article&k1=1625542155000000000&k2=565

In [17]:
import requests as req
from bs4 import BeautifulSoup
import datetime,random
import re

def randomtimes (start, end, n, frmt="%Y/%m/%d %H:%M" ) :
    stime=datetime.datetime.strptime(start,frmt)
    etime=datetime.datetime.strptime(end, frmt)
    time_datetime=[random.random() * (etime - stime) + stime for _ in range(n)]
    time_str=[t.strftime(frmt) for t in time_datetime]
    return time_str

print(randomtimes ('2011/01/01 00:00', '2011/12/31 00:00' , 1))
#randomtimes ('2012/01/01 00:00', '2012/12/31 00:00' , 10)
#randomtimes ('2013/01/01 00:00', '2013/12/31 00:00' , 10)
#randomtimes ('2014/01/01 00:00', '2014/12/31 00:00' , 10)
#randomtimes ('2015/01/01 00:00', '2015/12/31 00:00' , 10)
#randomtimes ('2016/01/01 00:00', '2016/12/31 00:00' , 10)
#randomtimes ('2017/01/01 00:00', '2017/12/31 00:00' , 10)
#randomtimes ('2018/01/01 00:00', '2018/12/31 00:00' , 10)
#randomtimes ('2019/01/01 00:00', '2019/12/31 00:00' , 10)

['2011/03/01 22:56']


In [47]:
import requests as req
from bs4 import BeautifulSoup
import re
import math
import random



userid =[]
url='https://www.pttweb.cc/user?sort=article'

#找10個ID的程式碼
def getuserID(url):
    l1 = random.sample(range(0,39),10)
    source=req.get(url).text
    root=BeautifulSoup(source,"lxml")
    for i in range(10):
        ids=root.find('div',{'class':'e7-wrapper mt-3'})
        id = ids.find_all('a')[l1[i]]['href']
        print(id)
        userid.append(id)
    print(userid)
getuserID(url)
#找此ID共發過多少文章>計算有多少頁>開始往每一頁去找特定年份的文章
posts_url='https://www.pttweb.cc'+userid[1]+'?t=article'
s=req.get(posts_url).text
r=BeautifulSoup(s,"lxml")
postnum=r.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
print(postnum)
postnum=postnum[postnum.index("共")+1:len(postnum)-1]
print(postnum)
pnum=int(postnum)
print(pnum)
for i in range(int(pnum/10)):
    nextpage_url=r.find(text='下一頁')
    nextpage_url=nextpage_url.getparent()
    

/user/allenchen98
/user/david7928
/user/laptic
/user/jeanvanjohn
/user/lpbrother
/user/filmwalker
/user/zkow
/user/hcc19880512
/user/jamescaesar
/user/kitune
['/user/allenchen98', '/user/david7928', '/user/laptic', '/user/jeanvanjohn', '/user/lpbrother', '/user/filmwalker', '/user/zkow', '/user/hcc19880512', '/user/jamescaesar', '/user/kitune']
作者 david7928 在 PTT 全部看板的發文, 共18568篇
18568
18568


AttributeError: 'NavigableString' object has no attribute 'getparent'

In [44]:
import requests as req
from bs4 import BeautifulSoup

posts_url='https://www.pttweb.cc/cavendishjr?t=article'
s=req.get(posts_url).text
r=BeautifulSoup(s,"lxml")
postnum=r.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
postnum=postnum[postnum.index("共")+1:len(postnum)-1]
print(postnum)
pnum=int(postnum)
print(pnum)

AttributeError: 'NoneType' object has no attribute 'text'

In [4]:
import requests as req
import lxml.html as lh
from bs4 import BeautifulSoup
#import re
#import math
#import random

posts_url='https://www.pttweb.cc/user/cavendishjr?t=article'
r = req.get(posts_url).text
soup = BeautifulSoup(r,"lxml")
results = soup.select("div.e7-pagination")
print(results)
#doc=lh.fromstring(posts_url)
#doc.xpath('//')
#print(doc)
#app=lh.tostring(doc.xpath('//@href/text()')).text
#print(app)
#print(lh.tostring(doc.xpath('//*[@id="app"]/div/main/div/div/div/div/div/div[1]/div/div[3]/div[3]/div/a[8]')[0]).decode())

nextpage_url=soup.select('a',{'data-v-c841833e':''},{'class':''})
print(nextpage_url)

[<div class="e7-pagination" data-v-c841833e=""><div class="left-rounded disabled" data-v-c841833e="">首頁</div><div class="disabled" data-v-c841833e="">上一頁</div><div class="e7-active" data-v-c841833e="">1</div><a data-v-c841833e="" href="/user/cavendishjr?t=article&amp;page=1"><div class="right-border" data-v-c841833e="">
                            2
                        </div></a><a data-v-c841833e="" href="/user/cavendishjr?t=article&amp;page=2"><div class="right-border" data-v-c841833e="">
                            3
                        </div></a><a data-v-c841833e="" href="/user/cavendishjr?t=article&amp;page=3"><div class="right-border" data-v-c841833e="">
                            4
                        </div></a><a data-v-c841833e="" href="/user/cavendishjr?t=article&amp;page=4"><div class="right-border" data-v-c841833e="">
                            5
                        </div></a><a data-v-c841833e="" href="/user/cavendishjr?t=article&amp;page=5"><div class="

TypeError: '<' not supported between instances of 'dict' and 'int'

In [24]:
import requests as req
from bs4 import BeautifulSoup
import re
import math

id=('hlyduck')
url='https://www.pttweb.cc/user/'+id+'?t=message'

source=req.get(url).text
root=BeautifulSoup(source,"lxml")
commentamount=root.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
commentamount=commentamount.split(id)
commentamount=re.sub("[^0-9]",'',commentamount[1])

pagenum=(int(commentamount))/10
pagenum=int(math.ceil(pagenum))

print(commentamount)
print(pagenum)

for i in range(pagenum):
    url='https://www.pttweb.cc/user/'+id+'?t=message&page='+str(i)
    source=req.get(url).text
    root=BeautifulSoup(source,"lxml")
    datas=root.find_all('span',{'class':'yellow--text text--darken-2'},{'data-v-4977f262':''})
    print(datas)
    print("")

37
4
[<span class="yellow--text text--darken-2" data-v-6184a124="">: 同學期修課路過幫推！不過我自己是覺得有些考試題目滿細</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 的，考前可能還是要花多一點時間把課本好好讀熟&gt; &lt;</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 燒</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: QQ</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 112</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 推推！</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 蓋</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: pui pui</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 啊啊啊</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 必須推</span>, <span class="yellow--text text--darken-2" data-v-6184a124="">: 欸XD</span>]

[<span class="yellow--text text--darken-2" data-v-6184a124="">: 有錢就推！</span>, <span class="yellow--text text--darken-2

In [4]:
import time
import random
a1=(1976,1,1,0,0,0,0,0,0)       #設定開始日期時間元組（1976-01-01 00：00：00）
a2=(1990,12,31,23,59,59,0,0,0)  #設定結束日期時間元組（1990-12-31 23：59：59）
start=time.mktime(a1)  #生成開始時間戳
end=time.mktime(a2)   #生成結束時間戳
#隨機生成10個日期字串
for i in range(10):
 t=random.randint(start,end)  #在開始和結束時間戳中隨機取出一個
 date_touple=time.localtime(t)     #將時間戳生成時間元組
 date=time.strftime("%Y/%m/%d",date_touple) #將時間元組轉成格式化字串（1976-05-21）
 print(date)

1981/04/04
1987/04/07
1989/02/15
1983/03/21
1983/03/03
1986/03/11
1989/04/30
1984/04/22
1986/09/30
1986/12/30


In [9]:
import requests as req
from bs4 import BeautifulSoup
import re
import math

url='https://www.pttweb.cc/newest/all?m=0'


for i in range(2):
    source=req.get(url).text
    root=BeautifulSoup(source,"lxml")
    datas=root.find_all('span',{'class':'text-no-wrap',{'data-v-3d41f38f':''}})
    print(datas)
    print("")

[<span class="text-no-wrap" data-v-3d41f38f="">4分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">4分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">5分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">5分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">5分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">5分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">5分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">5分鐘前</span>, <span class="text-no-wrap" data-v-3d41f38f="">09/24</span>, <span class="text-no-wrap" data-v-3d41f38f="">6

In [5]:
import requests
import time
from bs4 import BeautifulSoup

domain_url = 'https://www.ptt.cc'

def get_ppt_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}  # 紀錄cookies 是否年滿18歲
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text
    
def get_pageinfo(resdata,today):
    soup = BeautifulSoup(resdata, 'html5lib')
    #取得上一頁href
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    print(paging_div)
    prev_url = paging_div.find_all('a')[1]['href']
    print(prev_url)
    
    pptdata = []  # 儲存取得的文章資料
    date_divs = soup.find_all('div', 'r-ent')
    print(date_divs) #先抓取<div class="r-ent"></div>
    for d in date_divs:
        today_data = d.find('div', 'date').text.strip() == today
        print(today_data)
        if today_data : #判斷文章是否是今天發佈
           # 取得推文數
           push_count = d.find('div', 'nrec').text
           print(push_count)
           push_num = 0
           if push_count:
                try:
                    push_num = int(push_count)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2', ...
                    # 若不是, 不做任何事，push_num 保持為 0
                    if push_count == '爆':
                        push_num = 00
                    elif push_count.startswith('X'):
                        push_num = 99
           print("推文數",push_num)
           # 取得文章連結
           if d.find('a'):  # 有超連結，代表文章存在
                href = d.find('a')['href']
                print("標題連結",href)
                title = d.find('a').text
                print("標題",title)
                author = d.find('div', 'author').text if d.find('div', 'author') else '' #作者有可能是空的
                print("作者",author)
              
                pptdata.append({
                    'title': title,
                    'href': href,
                    'push_num': push_num,
                    'author': author
                })
           
    return pptdata,prev_url

if __name__ == '__main__':
    ppt_page = get_ppt_page(domain_url + '/bbs/Gossiping/index.html')
    if ppt_page:
        #print(domain_url + '/bbs/Gossiping/index.html')
        articles = []  # 全部的今日文章
        today = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        pptdata,prev_href = get_pageinfo(ppt_page,today)
        print(domain_url+prev_href)
        print(pptdata)

<div class="btn-group btn-group-paging">
				<a class="btn wide" href="/bbs/Gossiping/index1.html">最舊</a>
				<a class="btn wide" href="/bbs/Gossiping/index39354.html">‹ 上頁</a>
				<a class="btn wide disabled">下頁 ›</a>
				<a class="btn wide" href="/bbs/Gossiping/index.html">最新</a>
			</div>
/bbs/Gossiping/index39354.html
[<div class="r-ent">
			<div class="nrec"><span class="hl f2">8</span></div>
			<div class="title">
			
				<a href="/bbs/Gossiping/M.1632476440.A.7D5.html">[新聞] 法官開庭拿鐵棒敲自己頭直喊痛 認定是凶器打臉檢方</a>
			
			</div>
			<div class="meta">
				<div class="author">YunHung46</div>
				<div class="article-menu">
					
					<div class="trigger">⋯</div>
					<div class="dropdown">
						<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E6%96%B0%E8%81%9E%5D+%E6%B3%95%E5%AE%98%E9%96%8B%E5%BA%AD%E6%8B%BF%E9%90%B5%E6%A3%92%E6%95%B2%E8%87%AA%E5%B7%B1%E9%A0%AD%E7%9B%B4%E5%96%8A%E7%97%9B+%E8%AA%8D%E5%AE%9A%E6%98%AF%E5%87%B6%E5%99%A8%E6%89%93%E8%87%89%E6%AA%A2%E6%96%B9">搜尋同標題文章</